# indexing

## part 0

In [4]:
!pip install elasticsearch
!pip install autoawq
!pip install ragas

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
from pprint import pprint
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
import numpy as np
from elasticsearch.helpers import bulk
import json


In [7]:
es = Elasticsearch(f'https://sour-pugs-wash.loca.lt',timeout=60, max_retries=3, retry_on_timeout=True)

try:
    client_info = es.info()
    print('Connected to Elasticsearch!')
    pprint(client_info.body)
except Exception as e:
    print(f"Connection failed: {e}")

<ipython-input-7-21041b070a68>:1: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch(f'https://sour-pugs-wash.loca.lt',timeout=60, max_retries=3, retry_on_timeout=True)


Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': '_g8en_PbS5a2cPP2N9LEzQ',
 'name': '49a61ee328aa',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2024-08-05T10:05:34.233336849Z',
             'build_flavor': 'default',
             'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.11.1',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.15.0'}}


## part 1

In [8]:

index_name = "dataset_1"
MODEL_NAME = 'paraphrase-multilingual-mpnet-base-v2'
EMBEDDING_DIM = 768

settings = {
    "settings": {
        "analysis": {
            "char_filter": {
                "zero_width_spaces": {
                    "type": "mapping",
                    "mappings": ["\\u200C=>\\u0020"]
                }
            },
            "filter": {
                "persian_stop": {"type": "stop","stopwords": "_persian_"}
            },
            "analyzer": {
                "rebuilt_persian": {
                    "tokenizer": "standard",
                    "char_filter": ["zero_width_spaces"],
                    "filter": [
                        "lowercase",
                        "decimal_digit",
                        "arabic_normalization",
                        "persian_normalization",
                        "persian_stop"
                    ]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "نام": {"type": "text","analyzer": "rebuilt_persian"},
            "توضیحات": {"type": "text","analyzer": "rebuilt_persian"},
            "ادرس": {"type": "text","analyzer": "rebuilt_persian"},
            "بهترین زمان": {"type": "keyword"},
            "نوع": {"type": "keyword"},
            "تاریخچه": {"type": "text","analyzer": "rebuilt_persian"},
            "embedding":{
                "type":"dense_vector",
                "dims":EMBEDDING_DIM,
                "index":True,
                "similarity":"cosine"
            }
        }
    }

}


es.indices.delete(index=index_name, ignore_unavailable=True)
es.options(ignore_status=400).indices.create(index=index_name, body=settings)
print(f"Index '{index_name}' created successfully!")

model=SentenceTransformer(MODEL_NAME)

with open("/content/drive/MyDrive/NLPPDARA/output.json", "r", encoding="utf-8") as f:
    data = json.load(f)

def enhance_documents():
    for doc in data:

        context = " ".join([
            doc.get("نام", ""),
            doc.get("توضیحات", ""),
            doc.get("ادرس", ""),
            doc.get("بهترین زمان", ""),
            doc.get("تاریخچه", ""),
            doc.get("نوع", "")
        ])
        doc["embedding"] = model.encode(
            context,
            convert_to_tensor=False,
            show_progress_bar=False
        ).tolist()

        yield doc

actions = [
    {
        "_op_type": "index",
        "_index": index_name,
        "_id": doc["id"],
        "_source": doc
    } for doc in enhance_documents()
]

success, _ = bulk(es, actions)
print(f"تعداد اسناد وارد شده: {success}")


Index 'dataset_1' created successfully!


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


تعداد اسناد وارد شده: 30


In [9]:
index_name2 = "dataset_2"
settings2 = {
    "settings": {
        "analysis": {
            "char_filter": {
                "zero_width_spaces": {
                    "type": "mapping",
                    "mappings": ["\\u200C=>\\u0020"]
                }
            },
            "filter": {
                "persian_stop": {
                    "type": "stop",
                    "stopwords": "_persian_"
                }
            },
            "analyzer": {
                "rebuilt_persian": {
                    "tokenizer": "standard",
                    "char_filter": ["zero_width_spaces"],
                    "filter": [
                        "lowercase",
                        "decimal_digit",
                        "arabic_normalization",
                        "persian_normalization",
                        "persian_stop"
                    ]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "question": {
                "type": "text",
                "analyzer": "rebuilt_persian"
            },
            "answer": {
                "type": "text",
                "analyzer": "rebuilt_persian"
                },
            "embedding":{
                "type":"dense_vector",
                "dims":EMBEDDING_DIM,
                "index":True,
                "similarity":"cosine"
            }
    }

}
}
es.indices.delete(index=index_name2, ignore_unavailable=True)
es.options(ignore_status=400).indices.create(index=index_name2, body=settings2)
print(f"Index '{index_name2}' created successfully!")


with open("/content/drive/MyDrive/NLPPDARA/output1.2.json", "r", encoding="utf-8") as f:
    data2 = json.load(f)

def enhance_documentsQ():
    for doc in data2:

        context = " ".join([
            doc.get("question", ""),
            doc.get("answer", ""),
        ])

        doc["embedding"] = model.encode(context,convert_to_tensor=False,show_progress_bar=False).tolist()

        yield doc

actions = [
    {
        "_op_type": "index",
        "_index": index_name2,
        "_id": doc["id"],
        "_source": doc
    } for doc in enhance_documentsQ()
]

success, _ = bulk(es, actions,request_timeout=120)
print(f"تعداد اسناد وارد شده: {success}")

Index 'dataset_2' created successfully!


<ipython-input-9-9ebd6a9a0188>:81: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  success, _ = bulk(es, actions,request_timeout=120)


تعداد اسناد وارد شده: 150


## part2

In [10]:
index_name3 = "dataset_3"
settings3 = {
    "settings": {
        "analysis": {
            "char_filter": {
                "zero_width_spaces": {
                    "type": "mapping",
                    "mappings": ["\\u200C=>\\u0020"]
                }
            },
            "filter": {
                "persian_stop": {
                    "type": "stop",
                    "stopwords": "_persian_"
                }
            },
            "analyzer": {
                "rebuilt_persian": {
                    "tokenizer": "standard",
                    "char_filter": ["zero_width_spaces"],
                    "filter": [
                        "lowercase",
                        "decimal_digit",
                        "arabic_normalization",
                        "persian_normalization",
                        "persian_stop"
                    ]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "نام": {
                "type": "text",
                "analyzer": "rebuilt_persian"
            },
            "توضیحات": {
                "type": "text",
                "analyzer": "rebuilt_persian"
            },
            "embedding": {
                "type": "dense_vector",
                "dims": EMBEDDING_DIM,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}


es.indices.delete(index=index_name3, ignore_unavailable=True)
es.indices.create(index=index_name3, body=settings3)
print(f"Index '{index_name3}' created successfully!")


with open("/content/drive/MyDrive/NLPPDARA/output2.json", "r", encoding="utf-8") as f:
    data3 = json.load(f)
for doc in data3:
    if "نام " in doc:
        doc["نام"] = doc.pop("نام ")


def enhance_documents():

    for doc in data3:

        context = " ".join([
            doc.get("نام", ""),
            doc.get("توضیحات", "")
        ])

        doc["embedding"] = model.encode(context, convert_to_tensor=False, show_progress_bar=False).tolist()
        yield doc

actions = [
    {
        "_op_type": "index",
        "_index": index_name3,
        "_id": doc["id"],
        "_source": doc
    }
    for doc in enhance_documents()
]

success, _ = bulk(es, actions)
print(f"Number of documents indexed: {success}")


Index 'dataset_3' created successfully!
Number of documents indexed: 20


In [11]:
index_name4 = "dataset_4"
es.indices.delete(index=index_name4, ignore_unavailable=True)
es.options(ignore_status=400).indices.create(index=index_name4, body=settings2)
print(f"Index '{index_name4}' created successfully!")

with open("/content/drive/MyDrive/NLPPDARA/output2.1.json", "r", encoding="utf-8") as f:
    data4 = json.load(f)

def enhance_documents():
    for doc in data4:

        context = " ".join([
            doc.get("question", ""),
            doc.get("answer", ""),
        ])

        doc["embedding"] = model.encode(context,convert_to_tensor=False,show_progress_bar=False).tolist()

        yield doc

actions = [
    {
        "_op_type": "index",
        "_index": index_name4,
        "_id": doc["id"],
        "_source": doc
    } for doc in enhance_documents()
]

success, _ = bulk(es, actions)
print(f"تعداد اسناد وارد شده: {success}")

Index 'dataset_4' created successfully!
تعداد اسناد وارد شده: 151


## part 3

In [12]:

index_name5 = "dataset_5"
settings5 = {
    "settings": {
        "analysis": {
            "char_filter": {
                "zero_width_spaces": {
                    "type": "mapping",
                    "mappings": ["\\u200C=>\\u0020"]
                }
            },
            "filter": {
                "persian_stop": {
                    "type": "stop",
                    "stopwords": "_persian_"
                }
            },
            "analyzer": {
                "rebuilt_persian": {
                    "tokenizer": "standard",
                    "char_filter": ["zero_width_spaces"],
                    "filter": [
                        "lowercase",
                        "decimal_digit",
                        "arabic_normalization",
                        "persian_normalization",
                        "persian_stop"
                    ]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "نام شهر": {"type": "text","analyzer": "rebuilt_persian"},
            "معرفی": {"type": "text","analyzer": "rebuilt_persian"},
            "آب و هوا": {"type": "text","analyzer": "rebuilt_persian"},
            "دیدنی ها": {"type": "text","analyzer": "rebuilt_persian"},
            "سوغات": {"type": "text","analyzer": "rebuilt_persian"},
            "غذاهای محلی ": {"type": "text","analyzer": "rebuilt_persian"},
            "اقامت ": {"type": "text","analyzer": "rebuilt_persian"},
            "رستوران ها": {"type": "text","analyzer": "rebuilt_persian"},
            "حمل‌ و نقل عمومی": {"type": "text","analyzer": "rebuilt_persian"},
            "آداب و رسوم مردم": {"type": "text","analyzer": "rebuilt_persian"},
            "راهنمای سفر": {"type": "text","analyzer": "rebuilt_persian"},
            "embedding":{
                "type":"dense_vector",
                "dims":EMBEDDING_DIM,
                "index":True,
                "similarity":"cosine"
            }
        }
    }

}


es.indices.delete(index=index_name5, ignore_unavailable=True)
es.options(ignore_status=400).indices.create(index=index_name5, body=settings5)
print(f"Index '{index_name5}' created successfully!")

with open("/content/drive/MyDrive/NLPPDARA/output3.2.json", "r", encoding="utf-8") as f:
    data5 = json.load(f)

def enhance_documents():
    for doc in data5:
        context = " ".join([
            doc.get("نام شهر") or "",
            doc.get("معرفی") or "",
            doc.get("آب و هوا") or "",
            doc.get("دیدنی ها") or "",
            doc.get("سوغات ") or "",
            doc.get("غذاهای محلی ") or "",
            doc.get("اقامت") or "",
            doc.get("رستوران ها") or "",
            doc.get("حمل‌ و نقل عمومی") or "",
            doc.get("آداب و رسوم مردم ") or "",
            doc.get("راهنمای سفر") or ""
        ])

        doc["embedding"] = model.encode(context, convert_to_tensor=False, show_progress_bar=False).tolist()

        yield doc

actions = [
    {
        "_op_type": "index",
        "_index": index_name5,
        "_id": doc["id"],
        "_source": doc
    } for doc in enhance_documents()
]

success, _ = bulk(es, actions)
print(f"تعداد اسناد وارد شده: {success}")


Index 'dataset_5' created successfully!
تعداد اسناد وارد شده: 5


In [13]:
index_name6 = "dataset_6"
es.indices.delete(index=index_name6, ignore_unavailable=True)
es.options(ignore_status=400).indices.create(index=index_name6, body=settings2)
print(f"Index '{index_name6}' created successfully!")

with open("/content/drive/MyDrive/NLPPDARA/STANIRA1.json", "r", encoding="utf-8") as f:
    data6 = json.load(f)

def enhance_documents():
    for doc in data6:

        context = " ".join([
            doc.get("question", ""),
            doc.get("answer", ""),
        ])
        doc["embedding"] = model.encode(context,convert_to_tensor=False,show_progress_bar=False).tolist()

        yield doc

actions = [
    {
        "_op_type": "index",
        "_index": index_name6,
        "_id": doc["id"],
        "_source": doc
    } for doc in enhance_documents()
]

success, _ = bulk(es, actions,request_timeout=120)
print(f"تعداد اسناد وارد شده: {success}")

Index 'dataset_6' created successfully!


<ipython-input-13-94bee684e0e3>:29: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  success, _ = bulk(es, actions,request_timeout=120)


تعداد اسناد وارد شده: 287


In [14]:
index_name7 = "dataset_7"
es.indices.delete(index=index_name7, ignore_unavailable=True)
es.options(ignore_status=400).indices.create(index=index_name7, body=settings2)
print(f"Index '{index_name7}' created successfully!")

with open("/content/drive/MyDrive/NLPPDARA/output3.3.json", "r", encoding="utf-8") as f:
    data7 = json.load(f)

def enhance_documents():
    for doc in data7:

        context = " ".join([
            doc.get("question", ""),
            doc.get("answer", ""),
        ])
        doc["embedding"] = model.encode(context,convert_to_tensor=False,show_progress_bar=False).tolist()

        yield doc

actions = [
    {
        "_op_type": "index",
        "_index": index_name7,
        "_id": doc["id"],
        "_source": doc
    } for doc in enhance_documents()
]

success, _ = bulk(es, actions,request_timeout=240)
print(f"تعداد اسناد وارد شده: {success}")

Index 'dataset_7' created successfully!


<ipython-input-14-a40466d07510>:29: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  success, _ = bulk(es, actions,request_timeout=240)


تعداد اسناد وارد شده: 799


## Query

In [15]:

def elastic_search1(query: str) -> str:
    index_name = f"dataset_1"
    query_embedding = model.encode(query, convert_to_tensor=False).tolist()


    search_query = {
        "function_score":{
            "query":{"multi_match":{"query":query,"fields":["نام", "توضیحات", "ادرس", "بهترین زمان", "نوع", "تاریخچه"]}},
            "functions":[{"script_score": {"script": {"source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0","params": {"query_vector": query_embedding}}}}],
            "boost_mode":"multiply",
            "score_mode":"sum"}
        }
    response = es.search(
        index=index_name,
        body={
            "size": 2,
            "query": search_query,
            "_source": ["نام", "توضیحات", "ادرس", "بهترین زمان", "نوع", "تاریخچه"]
        }
    )

    if not response['hits']['hits']:
        return "نتیجهای یافت نشد"



    formatted_results = ""

    for i, hit in enumerate(response['hits']['hits'], start=1):
        result = hit['_source']
        formatted_results += (
            f"نام: {result.get('نام', '--')}\n"
            f"توضیحات: {result.get('توضیحات', '--')}\n"
            f"آدرس: {result.get('ادرس', '--')}\n"
            f"نوع: {result.get('نوع', '--')}\n"
            f"بهترین زمان بازدید: {result.get('بهترین زمان', '--')}\n"
            f"تاریخچه: {result.get('تاریخچه', '--')}\n")

    return formatted_results


In [16]:
def elastic_search2(query: str) -> str:
    index_name = f"dataset_2"
    query_embedding = model.encode(query, convert_to_tensor=False).tolist()

    search_query = {
        "function_score":{
            "query": {"multi_match": {"query":query,"fields":["question", "answer"]}},
            "functions":[{"script_score": {"script": {"source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0","params": {"query_vector": query_embedding}}}}],
            "boost_mode":"multiply",
            "score_mode":"sum"
    }
    }


    response = es.search(
        index=index_name,
        body={
            "size": 3,
            "query": search_query,
            "_source": ["question", "answer"]
        }
    )

    if not response['hits']['hits']:
        return "نتیجهای یافت نشد"

    formatted_results = ""
    for i, hit in enumerate(response['hits']['hits'], start=1):
        result = hit['_source']
        formatted_results += (
            f"Match {i}:\n"
            f"question: {result.get('question', '--')}\n"
            f"answer: {result.get('answer', '--')}\n\n"
        )

    return formatted_results

In [17]:
def elastic_search3(query: str) -> str:
    index_name = f"dataset_3"
    query_embedding = model.encode(query, convert_to_tensor=False).tolist()

    search_query = {
        "function_score": {
            "query": {
                "multi_match": {
                    "query": query,
                    "fields": ["نام^2", "توضیحات"]
                }
            },
            "functions": [
                {
                    "script_score": {
                        "script": {
                            "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                            "params": {"query_vector": query_embedding}
                        }
                    }
                }
            ],
            "boost_mode": "multiply",
            "score_mode": "sum"
        }
    }

    response = es.search(
        index=index_name,
        body={
            "size": 2,
            "query": search_query,
            "_source": ["نام", "توضیحات"]
        }
    )

    if not response['hits']['hits']:
        return "نتیجه ای یافت نشد"

    formatted_results = ""
    for i, hit in enumerate(response['hits']['hits'], start=1):
        result = hit['_source']
        formatted_results += (
            f"Match {i}:\n"
            f"نام: {result.get('نام', '--')}\n"
            f"توضیحات: {result.get('توضیحات', '--')}\n\n"
        )
    return formatted_results


In [18]:
def elastic_search4(query: str) -> str:
    index_name = f"dataset_4"
    query_embedding = model.encode(query, convert_to_tensor=False).tolist()

    search_query = {
        "function_score":{
            "query": {"multi_match": {"query":query,"fields":["question", "answer"]}},
            "functions":[{"script_score": {"script": {"source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0","params": {"query_vector": query_embedding}}}}],
            "boost_mode":"multiply",
            "score_mode":"sum"
    }
    }


    response = es.search(
        index=index_name,
        body={
            "size": 3,
            "query": search_query,
            "_source": ["question", "answer"]
        }
    )

    if not response['hits']['hits']:
        return "نتیجهای یافت نشد"

    formatted_results = ""
    for i, hit in enumerate(response['hits']['hits'], start=1):
        result = hit['_source']
        formatted_results += (
            f"Match {i}:\n"
            f"question: {result.get('question', '--')}\n"
            f"answer: {result.get('answer', '--')}\n\n"
        )

    return formatted_results

In [19]:
def elastic_search5(query: str) -> str:
    index_name = f"dataset_5"
    query_embedding = model.encode(query, convert_to_tensor=False).tolist()
    search_query = {
        "function_score":{
            "query": {"multi_match": {"query":query,"fields":["نام شهر","معرفی ","آب و هوا","دیدنی ها","سوغات ","غذاهای محلی ","اقامت ","رستوران ها","حمل‌ و نقل عمومی ", "آداب و رسوم مردم "]}},
            "functions":[{"script_score": {"script": {"source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0","params": {"query_vector": query_embedding}}}}],
            "boost_mode":"multiply",
            "score_mode":"sum"
    }
    }
    search_query = {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                "params": {"query_vector": query_embedding}
            }
        }
    }

    response = es.search(
        index=index_name,
        body={
            "size": 1,
            "query": search_query,
            "_source": ["نام شهر","معرفی ","آب و هوا","دیدنی ها","سوغات ",
                        "غذاهای محلی ","اقامت ","رستوران ها","حمل‌ و نقل عمومی ", "آداب و رسوم مردم ","راهنمای سفر "]
           }
    )

    if not response['hits']['hits']:
        return "نتیجهای یافت نشد"

    formatted_results = ""
    for i, hit in enumerate(response['hits']['hits'], start=1):
        result = hit['_source']
        formatted_results += (
            f"Match {i}:\n"
            f"نام شهر: {result.get('نام شهر', '--')}\n"
            f"معرفی : {result.get('معرفی', '--')}\n\n"
            f"آب و هوا {result.get('آب و هوا', '--')}\n\n"
            f"دیدنی ها: {result.get('دیدنی ها', '--')}\n\n"
            f"سوغات : {result.get('سوغات ', '--')}\n\n"
            f"غذاهای محلی : {result.get('غذاهای محلی ', '--')}\n\n"
            f"اقامت : {result.get('اقامت',' --')}\n\n"
            f"رستوران ها: {result.get('رستوران ها', '--')}\n\n"
            f"حمل‌ و نقل عمومی : {result.get('حمل‌ و نقل عمومی', '--')}\n\n"
            f"آداب و رسوم مردم : {result.get('آداب و رسوم مردم', '--')}\n\n"

        )

    return formatted_results

In [20]:
def elastic_search6(query: str) -> str:
    index_name = f"dataset_6"
    query_embedding = model.encode(query, convert_to_tensor=False).tolist()

    search_query = {
        "function_score":{
            "query": {"multi_match": {"query":query,"fields":["question", "answer"]}},
            "functions":[{"script_score": {"script": {"source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0","params": {"query_vector": query_embedding}}}}],
            "boost_mode":"multiply",
            "score_mode":"sum"
    }
    }


    response = es.search(
        index=index_name,
        body={
            "size": 3,
            "query": search_query,
            "_source": ["question", "answer"]
        }
    )

    if not response['hits']['hits']:
        return "نتیجهای یافت نشد"

    formatted_results = ""
    for i, hit in enumerate(response['hits']['hits'], start=1):
        result = hit['_source']
        formatted_results += (
            f"Match {i}:\n"
            f"question: {result.get('question', '--')}\n"
            f"answer: {result.get('answer', '--')}\n\n"
        )

    return formatted_results

In [21]:
def elastic_search7(query: str) -> str:
    index_name = f"dataset_7"
    query_embedding = model.encode(query, convert_to_tensor=False).tolist()

    search_query = {
        "function_score":{
            "query": {"multi_match": {"query":query,"fields":["question", "answer"]}},
            "functions":[{"script_score": {"script": {"source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0","params": {"query_vector": query_embedding}}}}],
            "boost_mode":"multiply",
            "score_mode":"sum"
    }
    }


    response = es.search(
        index=index_name,
        body={
            "size": 3,
            "query": search_query,
            "_source": ["question", "answer"]
        }
    )

    if not response['hits']['hits']:
        return "نتیجهای یافت نشد"

    formatted_results = ""
    for i, hit in enumerate(response['hits']['hits'], start=1):
        result = hit['_source']
        formatted_results += (
            f"Match {i}:\n"
            f"question: {result.get('question', '--')}\n"
            f"answer: {result.get('answer', '--')}\n\n"
        )

    return formatted_results

# Agentic RAG with Elasticsearch

In [22]:
from transformers import pipeline

classifier = pipeline("text-generation", model="Qwen/Qwen2-7B-Instruct-AWQ")


You have loaded an AWQ model on CPU and have a CUDA device available, make sure to set your model on a GPU device in order to run your model.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [29]:
def classify_dataset(query: str) -> int:
    messages = [
        {
            "role": "system",
            "content": """
            ### دستورالعمل:
            فقط عدد مربوط به دسته را برگردانید:
            1. سوالات  مربوط به اماکن تاریخی/فرهنگی ایران در هر شهری
            2. سوغاتی
            3. راهنمایی سفر به شهر های اصفهان ,مشهد، کیش ،رشت یا رامسر
            4. سوال درمورد سفر به شهر های استانبول یاایروان
            4. سوال درمورد سفر خارج از ایران به شرطی که سوال مربوط به  شهر استانبول یا ایروان نباشد
            5.  سوال های مانند در موضوعات بالا قرار ندارند

            مثال:


            کاربر: آثار تخت جمشید؟ -> 1
            کاربر: سوغاتی یزد چسیت؟ -> 2
            کاربر: فصل مناسب برای سفر به شهر مشهد چیه؟ -> 3
            کاربر: آیا ویزای شینگن برای سفر ضروری است؟ -> 4
            کاربر: چرا باید به شهر استانبول سفر کرد پاریس رو پیشنهاد میدی یا استانبول رو" -> 4
            کاربر: چطور به فضا سفر کنم یا تیم فوتبال رئال مادرید در کدوم شهر است؟ -> 5
            """
        },
        {"role": "user", "content": query}
    ]

    response = classifier(
        messages,
        max_new_tokens=1,
        top_p=0.1,
        num_return_sequences=1,
        pad_token_id=classifier.tokenizer.eos_token_id
    )

    answer = response[0]['generated_text'][-1]['content'].strip()

    try:
        return int(answer)
    except ValueError:
        import re
        match = re.search(r'\d', answer)
        return int(match.group()) if match else 0

def generate_answer(context: str, question: str,dataset_num:int) -> str:
    messages = [
        {
            "role": "system",
            "content": f" با استفاده از متن زیر به سوال پاسخ دهید ممکن است متن اطلاعات زیادی داشته باشد فقط به سوال بصورت کوتاه و دقیق بر اساس متن پاسخ بده:\n{context}"
        },
        {"role": "user", "content": question}
    ]

    response = classifier(
        messages,
        max_new_tokens=450,
        temperature=0.7,
        pad_token_id=classifier.tokenizer.eos_token_id
    )

    return response[0]['generated_text'][-1]['content']



def generate_answer1(context: str,context2: str, question: str,dataset_num:int) -> str:
    messages = [
        {
            "role": "system",
            "content": f"  با استفاده از متن زیر به سوال پاسخ دهید ممکن است متن اطلاعات زیادی داشته باشد فقط به سوال بصورت کوتاه و دقیق بر اساس متن پاسخ بده:\n{context}\n{context2}."
        },
        {"role": "user", "content": question}
    ]

    response = classifier(
        messages,
        max_new_tokens=450,
        temperature=0.7,
        pad_token_id=classifier.tokenizer.eos_token_id
    )

    return response[0]['generated_text'][-1]['content']

def full_pipeline(user_query: str) -> str:
    dataset_num = classify_dataset(user_query)
    print(f"\n Dataset detected: {dataset_num}")
    print(f"\n{user_query}")
    if dataset_num==1:
      try:
          context1 = elastic_search1(user_query)
          context2 = elastic_search2(user_query)
          contexts = [context1, context2]
          #print(f"Context found: {context1[:100]}...")
          #print(f"Context2 found: {context2[:100]}...")
          final_answer = generate_answer1(context1 ,context2, user_query,dataset_num)
      except Exception as e:

          return {"question": user_query,"contexts": [],"answer": f"خطا در پردازش: {str(e)}"}

    elif dataset_num==2:
      try:
          context3 = elastic_search3(user_query)
          context4 = elastic_search4(user_query)
          contexts = [context3, context4]
          #print(f"Context found: {context3[:100]}...")
          #print(f"Context2 found: {context4[:100]}...")
          final_answer = generate_answer1(context3 ,context4, user_query,dataset_num)
      except Exception as e:

          return {"question": user_query,"contexts": [],"answer": f"خطا در پردازش: {str(e)}"}

    elif dataset_num==3:
      try:
          context5 = elastic_search5(user_query)
          contexts = [context5]
          #print(f"Context found: {context5[:100]}...")
          final_answer = generate_answer(context5, user_query,dataset_num)
      except Exception as e:

          return {"question": user_query,"contexts": [],"answer": f"خطا در پردازش: {str(e)}"}


    elif dataset_num==4 :
      try:
          context6 = elastic_search6(user_query)
          context7 = elastic_search7(user_query)
          contexts = [context6, context7]
          #print(f"Context found: {context6[:100]}...")
          #print(f"Context2 found: {context7[:100]}...")
          final_answer = generate_answer1(context6 ,context6, user_query,dataset_num)
      except Exception as e:

            return {"question": user_query,"contexts": [],"answer": f"خطا در پردازش: {str(e)}"}
    else:
        print("\n اطلاعات کافی برای پاسخ به این سوال را ندارم")
        print("-------------------------------------------------------------------\n")

        return {"query": user_query, "contexts": [], "answer": "اطلاعات کافی برای پاسخ به این سوال را ندارم"}


    return { "question": user_query,"contexts": contexts,"answer": final_answer.content if hasattr(final_answer, 'content') else str(final_answer)}



In [41]:

test_queries = [
    ("ایا نقش جهان در شیراز است اگر نیست ادرسش کجاست؟"),
    ("شیرینی معروف یزد چیه؟"),
    ("تخت سلیمان مربوط به کدام دوره است؟"),
    ("ایا کلیسای نوتردام از مهم‌ترین دیدنی‌های پاریس هست؟"),
    ("چرا باید به شهر استانبول سفر کرد لندن رو پیشنهاد میدی یا استانبول رو؟"),
    ("مکان های تفریحی برای سفر به شهر رشت رو بگو؟"),
    ("سوغاتی مشهد چیست؟"),
    ("چه برای کاری روز های گرم تابستان مناسب است؟"),
    ("آثار تخت جمشید درکجا قرار دارد"),
]

for query in test_queries:
    answer=full_pipeline(query)
    print(answer["answer"])


 Dataset detected: 1

ایا نقش جهان در شیراز است اگر نیست ادرسش کجاست؟
نقش جهان در شهر اصفهان و نه شیراز قرار دارد. ادرس میدان نقش جهان در ایران، استان اصفهان، اصفهان است.

 Dataset detected: 2

شیرینی معروف یزد چیه؟
شیرینی معروف یزد "قطاب" است.

 Dataset detected: 1

تخت سلیمان مربوط به کدام دوره است؟
تخت سلیمان مربوط به دوره‌های مختلف تاریخی شامل اشکانی، ساسانی و ایلخانی است.

 Dataset detected: 4

ایا کلیسای نوتردام از مهم‌ترین دیدنی‌های پاریس هست؟
بله، کلیسای نوتردام یکی از مهم‌ترین دیدنی‌های پاریس است.

 Dataset detected: 4

چرا باید به شهر استانبول سفر کرد لندن رو پیشنهاد میدی یا استانبول رو؟
این بستگی به علت و مزیت شخصی شما دارد. لندن و استانبول دو شهر بزرگ و جذاب با دیدگاه های گوناگون هستند. لندن برای دانشجویان و محبوب هنرهای زیبا جذاب است، در حالی که استانبول برای افرادی که علاقه مند به تجربه های تاریخی و ثقافی هستند جذاب است. استانبول به عنوان یک رابطه بین شرق و غرب، دیدگاه های گوناگونی از فرهنگ، معماری و تاریخ را ارائه می دهد. لندن به عنوان مرکز فرهنگ و علمی جهان شناخته شده 

# Evaluation

In [ ]:
def generate_answer3(question: str) -> str:
    messages = [
        {
            "role": "system",
            "content": f"به سوال جواب بده"
        },
        {"role": "user", "content": question}
    ]

    response = classifier(
        messages,
        max_new_tokens=150,
        temperature=0.7,
        pad_token_id=classifier.tokenizer.eos_token_id
    )

    return response[0]['generated_text'][-1]['content']
test_queries = [
    ("ایا نقش جهان در شیراز است اگر نیست ادرسش کجاست؟"),
    ("شیرینی معروف یزد چیه؟"),
    ("تخت سلیمان مربوط به کدام دوره است؟"),
    ("ایا کلیسای نوتردام از مهم‌ترین دیدنی‌های پاریس هست؟"),
    ("چرا باید به شهر استانبول سفر کرد لندن رو پیشنهاد میدی یا استانبول رو؟"),
    ("مکان های تفریحی برای سفر به شهر رشت رو بگو؟"),
    ("سوغاتی مشهد چیست؟"),
    ("چه برای کاری روز های گرم تابستان مناسب است؟"),
    ("آثار تخت جمشید درکجا قرار دارد"),
]

for query in test_queries:
    answer=generate_answer3(query)
    print(f"\n {answer}\n")


 نقشه جهان در شیراز، ایران در محوطه بازار گلستان واقع شده است. اگر برای نقشه جهان به دنبال یک مکان خاص هستید، ممکن است بهتر باشد برای محل دقیق تر آن مراجعه کنید. به طور کلی، نقشه جهان یک مکان تاریخی و دینی است که در میان شهر شیراز قرار دارد.


 شیرینی معروف در یزد به "شیرینی سعیدی" یا "شیرینی نوری" می‌گویند. این دو شیرینی که در یزد قرار دارند، به خاطر طعم و فن آهنگی خود شهرت دارند و گاهی برای سالن های گردشگری و بازار محلی یزد، تولید می شوند.


 تخت سلیمان، که اغلب به عنوان یک عنصر داستانی در کتاب مقدس ذکر می شود، به دوران پادشاه سلیمان در عهد قدیم در ایران مربوط می شود. این دوران تقریباً بین سال ها 960 و 923 قبل از میلاد می روند.


 بله، کلیسای نوتردام یکی از مهم‌ترین دیدنی‌های پاریس است. این کلیسا به عنوان یکی از معماری‌های برجستهٔ میانه‌ای در فرانسه شناخته می‌شود و برای بسیاری از گردشگران و علاقه‌مندان به تاریخ و معماری یکی از نقاط زیارت اصلی است.


 در این قسمت می توان از دو شهر، استانبول و لندن، برای سفر انتخاب کرد و هر دو شهر دارای مزایای خود هستند:

لندن: 

1. تاریخ: لندن یکی از